In [10]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import scipy.stats
from prettytable import PrettyTable
%matplotlib inline
import math

In [2]:
dg = pd.read_csv('LRM_1_hour.csv')

In [3]:
len(dg.stool_day_total)

64

In [4]:
def logistic(X_train, X_test, y_train):
    model = LogisticRegression()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test) 
    return y_pred

In [5]:
cols = ['pulserate_SE',
       'pulserate_DFA', 'pulserate_ADF', 'pulserate_Mean', 'pulserate_Var',
       'ecg_resprate_SE', 'ecg_resprate_DFA', 'ecg_resprate_ADF',
       'ecg_resprate_Mean', 'ecg_resprate_Var', 'spo2_SE', 'spo2_DFA',
       'spo2_ADF', 'spo2_Mean', 'spo2_Var', 'heartrate_SE', 'heartrate_DFA',
       'heartrate_ADF', 'heartrate_Mean', 'heartrate_Var', 'peep_SE',
       'peep_DFA', 'peep_ADF', 'peep_Mean', 'peep_Var', 'pip_SE', 'pip_DFA',
       'pip_ADF', 'pip_Mean', 'pip_Var', 'map_SE', 'map_DFA', 'map_ADF',
       'map_Mean', 'map_Var', 'tidalvol_SE', 'tidalvol_DFA', 'tidalvol_ADF',
       'tidalvol_Mean', 'tidalvol_Var', 'minvol_SE', 'minvol_DFA',
       'minvol_ADF', 'minvol_Mean', 'minvol_Var', 'ti_SE', 'ti_DFA', 'ti_ADF',
       'ti_Mean', 'ti_Var', 'fio2_SE', 'fio2_DFA', 'fio2_ADF', 'fio2_Mean',
       'fio2_Var', 'abdomen_girth', 'urine', 'total_intake',
       'totalparenteralvolume', 'new_ph', 'gender', 'birthweight',
       'birthlength', 'birthheadcircumference', 'inout_patient_status',
        'baby_type', 'apgar_onemin',
       'apgar_fivemin', 'apgar_tenmin', 'motherage', 'conception_type',
       'mode_of_delivery', 'steroidname', 'numberofdose', 'rbs', 'temp',
       'currentdateweight', 'currentdateheight',  'gestation',
        'tpn-tfl', 'abd_difference', 'stool_day_total',
       'typevalue_Antibiotics', 'typevalue_Inotropes','mean_bp', 'sys_bp',
       'dia_bp']

In [6]:
def training(X,y):
    auc_roc = []
    training = []

    for i in range(5):
        try:
            X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                            test_size=0.3)
            # SMOTE
            sm = SMOTE(k_neighbors = 2)
            X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)
            y_smote = logistic(X_train_sm, X_test, y_train_sm)
            y_smote_1 = logistic(X_train_sm,X_train,y_train_sm)
            auc_roc.append(roc_auc_score(y_test,y_smote))
            training.append(roc_auc_score(y_train,y_smote_1))
            continue
        except Exception as e:
            continue
    return auc_roc,y_test,y_smote


            


In [7]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return round(m,3), round(m-h,3), round(m+h,3)

In [11]:
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_a = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_1 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

se_ppv = math.sqrt((ppv_1*(1-ppv_1))/(cm[0][0]+cm[0][1]))
ci_ppv_upper_1 = ppv_1 + (1.96*se_ppv)
ci_ppv_lower_1 = ppv_1 - (1.96*se_ppv)


se_npv = math.sqrt((npv_1*(1-npv_1))/(cm[1][0]+cm[1][1]))
ci_npv_upper_1 = npv_1 + (1.96*se_npv)
ci_npv_lower_1 = npv_1 - (1.96*se_npv)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [12]:
dg = pd.read_csv('LRM_6_hour.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_6 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_6 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_6 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

se_ppv = math.sqrt((ppv_6*(1-ppv_6))/(cm[0][0]+cm[0][1]))
ci_ppv_upper_6 = ppv_6 + (1.96*se_ppv)
ci_ppv_lower_6 = ppv_6 - (1.96*se_ppv)


se_npv = math.sqrt((npv_6*(1-npv_6))/(cm[1][0]+cm[1][1]))
ci_npv_upper_6 = npv_6 + (1.96*se_npv)
ci_npv_lower_6 = npv_6 - (1.96*se_npv)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [13]:
dg = pd.read_csv('LRM_12_hour.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_12 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_12 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_12 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

se_ppv = math.sqrt((ppv_12*(1-ppv_12))/(cm[0][0]+cm[0][1]))
ci_ppv_upper_12 = ppv_12 + (1.96*se_ppv)
ci_ppv_lower_12 = ppv_12 - (1.96*se_ppv)


se_npv = math.sqrt((npv_12*(1-npv_12))/(cm[1][0]+cm[1][1]))
ci_npv_upper_12 = npv_12 + (1.96*se_npv)
ci_npv_lower_12 = npv_12 - (1.96*se_npv)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [14]:
dg = pd.read_csv('LRM_48_hour.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_48 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_48 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_48 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

se_ppv = math.sqrt((ppv_48*(1-ppv_48))/(cm[0][0]+cm[0][1]))
ci_ppv_upper_48 = ppv_48 + (1.96*se_ppv)
ci_ppv_lower_48 = ppv_48 - (1.96*se_ppv)


se_npv = math.sqrt((npv_48*(1-npv_48))/(cm[1][0]+cm[1][1]))
ci_npv_upper_48 = npv_48 + (1.96*se_npv)
ci_npv_lower_48 = npv_48 - (1.96*se_npv)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [15]:
dg = pd.read_csv('LRM_1_week.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_1w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_1w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

se_ppv = math.sqrt((ppv_1w*(1-ppv_1w))/(cm[0][0]+cm[0][1]))
ci_ppv_upper_1w = ppv_1w + (1.96*se_ppv)
ci_ppv_lower_1w = ppv_1w - (1.96*se_ppv)


se_npv = math.sqrt((npv_1w*(1-npv_1w))/(cm[1][0]+cm[1][1]))
ci_npv_upper_1w = npv_1w + (1.96*se_npv)
ci_npv_lower_1w = npv_1w - (1.96*se_npv)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [16]:
dg = pd.read_csv('LRM_2_week.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_2w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_2w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_2w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

se_ppv = math.sqrt((ppv_2w*(1-ppv_2w))/(cm[0][0]+cm[0][1]))
ci_ppv_upper_2w = ppv_2w + (1.96*se_ppv)
ci_ppv_lower_2w = ppv_2w - (1.96*se_ppv)


se_npv = math.sqrt((npv_2w*(1-npv_2w))/(cm[1][0]+cm[1][1]))
ci_npv_upper_2w = npv_2w + (1.96*se_npv)
ci_npv_lower_2w = npv_2w - (1.96*se_npv)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [17]:
dg = pd.read_csv('LRM_3_week.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_3w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_3w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_3w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

se_ppv = math.sqrt((ppv_3w*(1-ppv_3w))/(cm[0][0]+cm[0][1]))
ci_ppv_upper_3w = ppv_3w + (1.96*se_ppv)
ci_ppv_lower_3w = ppv_3w - (1.96*se_ppv)


se_npv = math.sqrt((npv_3w*(1-npv_3w))/(cm[1][0]+cm[1][1]))
ci_npv_upper_3w = npv_3w + (1.96*se_npv)
ci_npv_lower_3w = npv_3w - (1.96*se_npv)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [18]:
dg = pd.read_csv('LRM_4_week.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_4w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_4w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_4w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

se_ppv = math.sqrt((ppv_4w*(1-ppv_4w))/(cm[0][0]+cm[0][1]))
ci_ppv_upper_4w = ppv_4w + (1.96*se_ppv)
ci_ppv_lower_4w = ppv_4w - (1.96*se_ppv)


se_npv = math.sqrt((npv_4w*(1-npv_4w))/(cm[1][0]+cm[1][1]))
ci_npv_upper_4w = npv_4w + (1.96*se_npv)
ci_npv_lower_4w = npv_4w - (1.96*se_npv)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [19]:



l = [["1 Hour" ,c_a, ppv_1,ci_ppv_upper_1,ci_ppv_lower_1,npv_1,ci_npv_upper_1,ci_npv_lower_1],["6 Hour" ,c_6, ppv_6,ci_ppv_upper_6,ci_ppv_lower_6,npv_6,ci_npv_upper_6,ci_npv_lower_6],["12 Hour" ,c_12, ppv_12,ci_ppv_upper_12,ci_ppv_lower_12,npv_12,ci_npv_upper_12,ci_npv_lower_12],["48 Hour" ,c_48, ppv_48,ci_ppv_upper_48,ci_ppv_lower_48,npv_48,ci_npv_upper_48,ci_npv_lower_48],["1 week" ,c_1w, ppv_1w,ci_ppv_upper_1w,ci_ppv_lower_1w,npv_1w,ci_npv_upper_1w,ci_npv_lower_1w],["2 Week" ,c_2w, ppv_2w,ci_ppv_upper_2w,ci_ppv_lower_2w,npv_2w,ci_npv_upper_2w,ci_npv_lower_2w],["3 Week" ,c_3w, ppv_3w,ci_ppv_upper_3w,ci_ppv_lower_3w,npv_3w,ci_npv_upper_3w,ci_npv_lower_3w],["4 Week" ,c_4w, ppv_4w,ci_ppv_upper_4w,ci_ppv_lower_4w,npv_4w,ci_npv_upper_4w,ci_npv_lower_4w]]


table = PrettyTable(['Parameter', 'AUC-ROC', 'PPV','PPV(Upper)','PPV(Lower)','NPV','NPV(Upper)','NPV(Lower)'])
for rec in l:
    table.add_row(rec)

print(table)

+-----------+-----------------------+------+--------------------+--------------------+------+--------------------+---------------------+
| Parameter |        AUC-ROC        | PPV  |     PPV(Upper)     |     PPV(Lower)     | NPV  |     NPV(Upper)     |      NPV(Lower)     |
+-----------+-----------------------+------+--------------------+--------------------+------+--------------------+---------------------+
|   1 Hour  | (0.867, 0.771, 0.963) | 0.75 |       0.995        |       0.505        | 0.88 | 1.1051868557442908 |  0.6548131442557092 |
|   6 Hour  | (0.782, 0.647, 0.917) | 0.85 | 1.0441062357814161 | 0.6558937642185837 | 0.43 | 0.7967572494171042 |  0.0632427505828958 |
|  12 Hour  | (0.848, 0.775, 0.921) | 0.85 | 1.0441062357814161 | 0.6558937642185837 | 1.0  |        1.0         |         1.0         |
|  48 Hour  | (0.948, 0.851, 1.045) | 1.0  |        1.0         |        1.0         | 0.62 | 0.956355645113918  |  0.283644354886082  |
|   1 week  |  (0.813, 0.756, 0.87) | 0.9